# Pips

In [ ]:
!pip install numpy pandas yfinance matplotlib scipy qiskit==2.0.0 qiskit-aer==0.17.0 qiskit-ibm-runtime==0.38.0 qiskit_algorithms==0.3.1 'qiskit[visualization]'


# Librerias

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes
from qiskit_aer.primitives import EstimatorV2
from qiskit_aer.primitives import SamplerV2

# Obtener los valores de Yahoo Finance

In [ ]:
def get_ibex35_data():
    tickers = [
        "ANA.MC", "ACX.MC", "ACS.MC", "AENA.MC", "ALM.MC", "AMS.MC",
        "BBVA.MC", "CABK.MC", "CLNX.MC", "DIA.MC", "EBRO.MC", "ELE.MC",
        "ENG.MC", "FER.MC", "GRF.MC", "IBE.MC", "IDR.MC", "ITX.MC",
        "COL.MC", "LOG.MC", "MAP.MC", "MEL.MC", "MRL.MC", "NTGY.MC",
        "PHM.MC", "REP.MC", "ROVI.MC", "SAB.MC", "SAN.MC"
    ]

    data = yf.download(
        tickers,
        start='2023-01-01',
        end='2024-04-01',
        auto_adjust=False
    )['Adj Close']

    # Manejo robusto de datos faltantes
    data = data.dropna(axis=1, how='all').ffill().bfill().fillna(0)

    returns = data.pct_change().dropna()
    mu = returns.mean().values * 252
    sigma = returns.cov().values * 252

    # Forzar matriz de covarianza a ser definida positiva y real
    sigma = (sigma + sigma.T) / 2
    sigma = np.real(sigma)
    sigma += np.eye(sigma.shape[0]) * 1e-6

    # Asegurarse de que no haya valores NaN o complejos
    mu = np.nan_to_num(mu, nan=0.0)
    sigma = np.nan_to_num(sigma, nan=0.0)

    return mu, sigma, data.columns.tolist()

# Hacemos nuestra ecuacion

In [ ]:
def build_portfolio_problem(mu, sigma, num_assets, budget=5, risk_factor=0.5):
    # Creamos un diccionario para representar el problema en formato QUBO
    qubo_dict = {}

    # Términos lineales (retornos esperados)
    for i in range(num_assets):
        qubo_dict[(i, i)] = float(-mu[i])

    # Términos cuadráticos (riesgo)
    for i in range(num_assets):
        for j in range(i+1, num_assets):
            qubo_dict[(i, j)] = float(risk_factor * sigma[i, j])

    # Restricción de presupuesto (seleccionar exactamente 'budget' acciones)
    # Implementamos como penalización cuadrática
    penalty = 10.0  # Factor de penalización alto

    # Añadir término (sum(x_i) - budget)^2
    for i in range(num_assets):
        qubo_dict[(i, i)] += penalty * (1 - 2*budget/num_assets)
    for i in range(num_assets):
        for j in range(i+1, num_assets):
            if (i, j) in qubo_dict:
                qubo_dict[(i, j)] += penalty * 2/num_assets**2
            else:
                qubo_dict[(i, j)] = penalty * 2/num_assets**2

    # Convertir QUBO a operador Hamiltoniano
    pauli_terms = []
    coeffs = []

    # Términos lineales y cuadráticos
    for (i, j), coeff in qubo_dict.items():
        if i == j:  # Término lineal
            pauli_str = ['I'] * num_assets
            pauli_str[i] = 'Z'
            pauli_terms.append(''.join(pauli_str))
            coeffs.append(float(coeff))
        else:  # Término cuadrático
            pauli_str = ['I'] * num_assets
            pauli_str[i] = 'Z'
            pauli_str[j] = 'Z'
            pauli_terms.append(''.join(pauli_str))
            coeffs.append(float(coeff))

    # Crear el operador hamiltoniano con coeficientes reales
    hamiltonian = SparsePauliOp(pauli_terms, coeffs)

    return hamiltonian, qubo_dict

# Vector Quantum Estimator

In [ ]:
class ManualVQE:
    def __init__(self, ansatz, estimator, maxiter=100):
        self.ansatz = ansatz
        self.estimator = estimator
        self.backend = AerSimulator(
                              method='statevector',
                              device='GPU',
                              cuStateVec_enable=True
                              )
        self.maxiter = maxiter
        self.cost_function_evals = 0
        self.optimizer_evals = 0

    def compute_minimum_eigenvalue(self, operator, initial_point=None):
        if initial_point is None:
            initial_point = np.random.random(self.ansatz.num_parameters) * 2 * np.pi

        self.cost_function_evals = 0

        def cost_function(params):
            self.cost_function_evals += 1
            circuit = self.ansatz.assign_parameters(params)

           # Usar self.backend en lugar de backend
            transpiled_circuit = transpile(circuit, self.backend)

          # Formato correcto para EstimatorV2
            job = self.estimator.run([(transpiled_circuit, operator)])
            result = job.result()
            return result.data.evs[0]  # Cambio clave aquí



        # Usar scipy.optimize para minimizar la función de costo
        from scipy.optimize import minimize
        result = minimize(
            cost_function,
            initial_point,
            method='COBYLA',
            options={'maxiter': 100}
        )

        self.optimizer_evals = result.nfev

        # Crear un objeto de resultado similar al de VQE original
        class VQEResult:
            def __init__(self, eigenvalue, optimal_point, cost_function_evals, optimizer_evals):
                self.eigenvalue = eigenvalue
                self.optimal_point = optimal_point
                self.cost_function_evals = cost_function_evals
                self.optimizer_evals = optimizer_evals

        return VQEResult(result.fun, result.x, self.cost_function_evals, self.optimizer_evals)

# Calculo de función de coste

In [ ]:
def calculate_portfolio_energy(bitstring, qubo_dict, num_assets):
    """Calcula la energía (costo) de un portafolio dado un bitstring."""
    energy = 0

    # Convertir bitstring a configuración binaria
    if isinstance(bitstring, int):
        binary = format(bitstring, f'0{num_assets}b')
        config = [int(bit) for bit in binary]
    else:
        config = [int(bit) for bit in bitstring]

    # Calcular términos lineales
    for i in range(num_assets):
        if (i, i) in qubo_dict and config[i] == 1:
            energy += qubo_dict[(i, i)]

    # Calcular términos cuadráticos
    for i in range(num_assets):
        for j in range(i+1, num_assets):
            if (i, j) in qubo_dict and config[i] == 1 and config[j] == 1:
                energy += qubo_dict[(i, j)]

    return energy

# Ejecución del circuito

In [ ]:
def main():
    # Obtener datos del IBEX35
    mu, sigma, tickers = get_ibex35_data()
    num_assets = len(tickers)

    # Construir el hamiltoniano para el problema
    hamiltonian, qubo_dict = build_portfolio_problem(mu, sigma, num_assets, budget=5, risk_factor=0.5)

    # Verificación final para asegurarse de que no haya coeficientes complejos
    coeffs = np.array(hamiltonian.coeffs, dtype=float)
    if not np.all(np.isreal(coeffs)):
        print("Advertencia: Se detectaron coeficientes complejos, tomando solo la parte real")
        coeffs = np.real(coeffs)
        hamiltonian = SparsePauliOp(hamiltonian.paulis, coeffs)

    # Configurar un simulador local para pruebas
    backend = AerSimulator()

# Crear un ansatz para VQE
    vqe_ansatz = RealAmplitudes(num_assets, reps=3, entanglement='linear')

# Número de parámetros en el circuito
    num_params = vqe_ansatz.num_parameters
    print(f"Número de parámetros del ansatz VQE: {num_params}")

# Configurar el estimador para VQE correctamente
    estimator = EstimatorV2.from_backend(backend)

# Inicializar parámetros aleatorios
    initial_point = np.random.random(num_params) * 2 * np.pi

    # Reemplazar todo el bloque de código de VQE por:
    vqe_algorithm = ManualVQE(
        ansatz=vqe_ansatz,
        estimator=estimator,
        maxiter=100
    )

    # Ejecutar el VQE para encontrar el autovector del ansatz
    print("Iniciando optimización VQE...")
    vqe_result = vqe_algorithm.compute_minimum_eigenvalue(hamiltonian, initial_point)

    # Obtener el resultado
    optimal_value = vqe_result.eigenvalue
    optimal_params = vqe_result.optimal_point

    print("Optimización completada:")
    print(f"Valor mínimo encontrado (energía): {optimal_value:.6f}")
    print(f"Número de evaluaciones de función: {vqe_result.cost_function_evals}")
    print(f"Número de iteraciones: {vqe_result.optimizer_evals}")


    # Ejecutar el circuito con los parámetros optimizados
    bound_circuit = vqe_ansatz.assign_parameters(optimal_params)
    qc_with_measurements = bound_circuit.copy()
    qc_with_measurements.measure_all()

    # Por esto:
    from qiskit_aer.primitives import SamplerV2
    sampler = SamplerV2.from_backend(backend)
    job = sampler.run([(transpile(qc_with_measurements, backend), 10000)])
    result = job.result()
    counts = result.data.meas[0].get_counts()
    # Ordenar las configuraciones por frecuencia (de mayor a menor)
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

    # Mostrar las 3 mejores opciones
    print("\n🔮 Top 3 Portafolios Cuánticos (VQE):")
    for i, (bitstring, probability) in enumerate(sorted_counts[:3]):
        # Convertir a representación binaria
        binary_string = format(bitstring, f'0{num_assets}b')

        # Obtener índices de acciones seleccionadas
        selected_indices = [j for j, bit in enumerate(binary_string) if bit == '0']
        selected_stocks = [tickers[j] for j in selected_indices]

        # Calcular energía para esta configuración
        energy = calculate_portfolio_energy(binary_string, qubo_dict, num_assets)

        print(f"\nOpción #{i+1} (Probabilidad: {probability:.4f}, Energía: {energy:.6f}):")
        print(f"Acciones seleccionadas: {selected_stocks}")

        # Calcular retorno esperado y riesgo
        portfolio_return = sum(mu[j] for j in selected_indices)

        # Calcular riesgo
        portfolio_risk = 0
        for j in selected_indices:
            for k in selected_indices:
                portfolio_risk += sigma[j, k]

        print(f"Retorno Esperado: {portfolio_return:.4f}")
        print(f"Riesgo: {portfolio_risk:.4f}")
        print(f"Ratio Retorno/Riesgo: {portfolio_return/portfolio_risk if portfolio_risk != 0 else 'N/A':.4f}")


In [ ]:
if __name__ == "__main__":
    main()

[*********************100%***********************]  29 of 29 completed
<ipython-input-7-9b4043b180aa>:10: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(hamiltonian.coeffs, dtype=float)


Número de parámetros del ansatz VQE: 116
Iniciando optimización VQE...
